In [2]:
import os
from dotenv import load_dotenv

In [11]:
env_path = os.path.join('envs', '.params.env')

if os.path.exists(env_path):
    load_dotenv(env_path)
    
    TRAIN_DATA_CSV = os.getenv('TRAIN_DATA_PATH')
    TEST_DATA_CSV = os.getenv('TEST_DATA_PATH')
    SEED = int(os.getenv('MODEL_PARAMETR_1'))
    max_iter_lr = int(os.getenv('max_iter_lr')) # для Логистической регрессии
    n_estimators_rf = int(os.getenv('n_estimators_rf')) # для Случайного леса
    EPOCHS = int(os.getenv('EPOCHS')) # для нейронной сети
    HIDDEN_SIZE = int(os.getenv('MODEL_PARAMETER_1')) # для нейронной сети
    
    print("Константы успешно загружены.")
else:
    print("Файл .params.env не найден!")

Константы успешно загружены.


In [4]:
import pandas as pd

In [5]:
train_df = pd.read_csv(TRAIN_DATA_CSV)
test_df = pd.read_csv(TEST_DATA_CSV)

print(f"Тренировочный файл '{TRAIN_DATA_CSV}' загружен.")
print(f"Тестовый файл '{TEST_DATA_CSV}' загружен.")

Тренировочный файл 'data/train.csv' загружен.
Тестовый файл 'data/eval.csv' загружен.


In [6]:
def preprocess(df):
    cols = ['survived', 'sex', 'age', 'n_siblings_spouses', 'parch', 'fare']
    df = df[cols].copy()
    
    df['age'] = df['age'].fillna(df['age'].median())
    df['sex'] = df['sex'].map({'male': 0, 'female': 1})
         
    return df

In [7]:
train_data = preprocess(train_df)
test_data = preprocess(test_df)

In [8]:
print("Пример тренировочных данных:")
train_data.head()

Пример тренировочных данных:


,survived,sex,age,n_siblings_spouses,parch,fare
0,0,0,22.0,1,0,7.2500
1,1,1,38.0,1,0,71.2833
2,1,1,26.0,0,0,7.9250
3,1,1,35.0,1,0,53.1000
4,0,0,28.0,0,0,8.4583


In [9]:
print("Данные готовы к обучению!")

Данные готовы к обучению!


In [14]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [17]:
X_train = train_data.drop('survived', axis=1)
y_train = train_data['survived']

X_test = test_data.drop('survived', axis=1)
y_test = test_data['survived']

In [18]:
print("\nОбучение Модели 1: Логистическая регрессия")
model_lr = LogisticRegression(random_state=SEED, max_iter=max_iter_lr)
model_lr.fit(X_train, y_train)

y_pred_lr = model_lr.predict(X_test)
accuracy_lr = accuracy_score(y_test, y_pred_lr)
print(f" Точность на test: {accuracy_lr:.4f}")


Обучение Модели 1: Логистическая регрессия
 Точность на test: 0.7500


In [50]:
from sklearn.ensemble import RandomForestClassifier

In [51]:
print("\nОбучение Модели 2: Случайный лес")
model_rf = RandomForestClassifier(n_estimators=n_estimators_rf, random_state=SEED)
model_rf.fit(X_train, y_train)

y_pred_rf = model_rf.predict(X_test)
accuracy_rf = accuracy_score(y_test, y_pred_rf)
print(f" Точность на test: {accuracy_rf:.4f}")


Обучение Модели 2: Случайный лес


 Точность на test: 0.8333


In [10]:
import torch
import torch.nn as nn

In [21]:
# превращение данных в тензоры
X_train_t = torch.tensor(X_train.values, dtype=torch.float32)
y_train_t = torch.tensor(y_train.values, dtype=torch.float32)
X_test_t = torch.tensor(X_test.values, dtype=torch.float32)
y_test_t = torch.tensor(y_test.values, dtype=torch.float32)

In [12]:
print("\nОбучение Модели 3: Нейронная сеть с одним скрытым слоем")
class SimpleNet(nn.Module):
    def __init__(self, in_features):
        super().__init__()
        self.fc1 = nn.Linear(in_features, HIDDEN_SIZE)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(HIDDEN_SIZE, 1)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x).squeeze(1)
        return x


Обучение Модели 3: Нейронная сеть с одним скрытым слоем


In [19]:
# Инициализация модели, функции потерь и оптимизатора
input_dim = X_train.shape[1]
model = SimpleNet(input_dim)
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [47]:
# Тренировочный цикл
for epoch in range(EPOCHS):
    model.train()
    optimizer.zero_grad()
    logits = model(X_train_t)
    loss = criterion(logits, y_train_t)
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 10 == 0:
        model.eval()
        with torch.no_grad():
            val_logits = model(X_test_t)
            val_probs = torch.sigmoid(val_logits)
            val_pred_labels = (val_probs.numpy() >= 0.5).astype(int)
            val_acc = accuracy_score(y_test_t, val_pred_labels)
        print(f"Epoch {epoch+1}/{EPOCHS} | loss={loss.item():.4f} | val_acc={val_acc:.4f}")

# Оценка на тесте
print(f"Точность на test: {val_acc:.4f}")

Epoch 10/50 | loss=0.4358 | val_acc=0.7424
Epoch 20/50 | loss=0.4355 | val_acc=0.7424
Epoch 30/50 | loss=0.4353 | val_acc=0.7424
Epoch 40/50 | loss=0.4350 | val_acc=0.7424
Epoch 50/50 | loss=0.4347 | val_acc=0.7462
Точность на test: 0.7462
